In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

import torch
torch.set_float32_matmul_precision('high')

In [2]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [ ]:
import numpy as np
import figs.utilities.config_helper as ch
import figs.utilities.transform_helper as th
from figs.control.vehicle_rate_uqp import VehicleRateUQP
import sousvide.visualize.plot_time as pt
import sousvide.visualize.plot_3D as p3d
import matplotlib.pyplot as plt

np.random.seed(0)
mu_md = np.zeros(10)
std_md = np.array([
            0.40, 0.40, 0.40,
            0.40, 0.40, 0.40,
            0.20, 0.20, 0.20, 0.20
            ])

policy = ch.get_config("vrfqp_fr_v1","pilots")
# policy = ch.get_config("vrfqp_fr_v2","pilots")
# course = ch.get_config("robustness_track","courses")
course = ch.get_config("line","courses")
# course = ch.get_config("bounce1","courses")
# course = ch.get_config("bounce2","courses")

test = VehicleRateUQP(policy,course)
p3d.tXU_to_3D([test.tXUd])

RuntimeError: Factor is exactly singular

In [ ]:
# policy = ch.get_config("vrfqp_fr_v1","pilots")
policy = ch.get_config("vrfqp_fr_v2","pilots")
# course = ch.get_config("robustness_track","courses")
# course = ch.get_config("bounce1","courses")
course = ch.get_config("bounce2","courses")

test = VehicleRateUQP(policy,course)
# p3d.tXU_to_3D([test.tXUd])

In [ ]:
Ts,FO = test.Ts,test.FO

p3d.